<a href="https://colab.research.google.com/github/xesmaze/cpsc541-fall2024/blob/main/lectures/ExpectationMaximization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The Expectation-Maximization (EM) algorithm is a powerful statistical technique used to estimate parameters in models with latent (unobserved) variables. It works iteratively by alternating between an E-step (Expectation) and an M-step (Maximization) until the estimates converge.

The classic example for EM algorithm demo is often the coin toss by two trick coins example- however, I realized that this example is not really resonating with a life sciences audiance. So instead we will use a breeding pairs example.

Let’s walk through the EM algorithm using a breeding-pairs mixed progeny record example, where we estimate the genetic contribution to traits using incomplete or hidden data, like unknown parentage or unknown genotypes of the progeny.

## **Problem Setup**

Consider a scenario where we have a population of progeny (offspring) resulting from several breeding pairs.

However, we don't fully know which progeny comes from which pair, or there are some unknowns about the genotypes or phenotypes of the parents.

Imagine that for each breeding pair, the progeny population is influenced by a gender bias.

This could mean that certain breeding pairs are more likely to produce male or female offspring. However, the specific assignment of progeny to each breeding pair is unknown, and the goal is to infer this assignment based on observed gender bias.

### **Observed Data**
- Gender (male/female) of each progeny.
- Total number of progeny produced.
- Gender bias probabilities for each breeding pair (e.g., breeding pair 1 produces 70% male offspring, pair 2 produces 40% male offspring, etc.).

### **Latent (Unobserved) Data:**
The assignment of progeny to breeding pairs.


To solve the problem of identifying which progeny belongs to which breeding pair based on gender bias using the Expectation-Maximization (EM) algorithm, we can break down the steps programmatically.

First- lets start by simplifying the problem

### **Assumptions & Process Steps**

- We have 3 breeding pairs with known gender biases.
- We know the gender of each progeny but not which pair they came from.
- We also know that each mating pair have a different gender bias for producing progeny.
- We will iterate the EM steps to update the probabilities of progeny assignments.

1. Initialization

  - Set initial probabilities for gender biases
  - Randomly assign initial probabilities for progeny-pair assignments.

2. Expectation Step

  - Calculate the probability that each progeny belongs to each pair based on their gender and current gender bias estimates.
  
3. Maximization Step

  - Update the gender bias estimates and the proportion of progeny assigned to each pair.

4. Iteration

 - Repeat E-step and M-step until convergence to the values that was implied for the breeding pairs.

In [ ]:
import numpy as np

# Example data
num_progeny = 100
num_pairs = 3

# Gender bias for each pair (initial known probabilities)
# e.g., Pair 1: 70% male, Pair 2: 40% male, Pair 3: 50% male
gender_bias = np.array([[0.7, 0.3],  # Pair 1: [P(male), P(female)]
                        [0.4, 0.6],  # Pair 2: [P(male), P(female)]
                        [0.5, 0.5]]) # Pair 3: [P(male), P(female)]

# Gender of progeny: 1 for male, 0 for female
progeny_gender = np.random.choice([1, 0], size=num_progeny, p=[0.5, 0.5])


# Equal probability initialization: each progeny is equally likely to belong to any pair
assignments = np.full((num_progeny, num_pairs), 1 / num_pairs)

# Or alternatively- we can randomly assign probabilities to belong to any given pair
# This is done with Dirichlet Distribution
# assignments = np.random.dirichlet(np.ones(num_pairs), num_progeny)
# The choice of initialization affects how the algorithm converges but shouldn't
# drastically impact the final result as long as the algorithm runs enough iterations to stabilize.

# E-step: Compute the probability that each progeny to belong to each pair
def e_step(progeny_gender, assignments, gender_bias):
    for i in range(num_progeny):
        for k in range(num_pairs):
            if progeny_gender[i] == 1:  # Male progeny
                assignments[i, k] = gender_bias[k, 0]  # P(male | pair k)
            else:  # Female progeny
                assignments[i, k] = gender_bias[k, 1]  # P(female | pair k)
        # Normalize to ensure the probabilities sum to 1
        assignments[i, :] /= np.sum(assignments[i, :])
    return assignments

# M-step: Update the gender bias and proportion estimates based on assignments
def m_step(assignments, progeny_gender):
    updated_gender_bias = np.zeros_like(gender_bias)

    # Update gender bias for each pair
    for k in range(num_pairs):
        total_male = np.sum(assignments[:, k] * (progeny_gender == 1))
        total_female = np.sum(assignments[:, k] * (progeny_gender == 0))
        total_progeny = total_male + total_female

        updated_gender_bias[k, 0] = total_male / total_progeny  # P(male | pair k)
        updated_gender_bias[k, 1] = total_female / total_progeny  # P(female | pair k)

    return updated_gender_bias

# EM Algorithm Loop
def em_algorithm(progeny_gender, assignments, gender_bias, max_iter=100, tol=1e-6):
    for iteration in range(max_iter):
        # E-step: Calculate the expected assignment probabilities
        assignments = e_step(progeny_gender, assignments, gender_bias)

        # M-step: Update the gender bias estimates
        new_gender_bias = m_step(assignments, progeny_gender)

        # Check for convergence
        if np.max(np.abs(gender_bias - new_gender_bias)) < tol:
            print(f"Converged after {iteration} iterations")
            break

        gender_bias = new_gender_bias

    return assignments, gender_bias

# Run the EM algorithm
final_assignments, final_gender_bias = em_algorithm(progeny_gender, assignments, gender_bias)

# Results: Final assignment probabilities and gender biases
print("Final Gender Bias Estimates:")
print(final_gender_bias)

#print("Final Progeny Assignments (Probabilities):")
#print(final_assignments)


Converged after 3 iterations
Final Gender Bias Estimates:
[[0.69699326 0.30300674]
 [0.39657842 0.60342158]
 [0.49643077 0.50356923]]


You can make the algorithm run longer for better estimates by lowering the tolerance threshold `tol`

In [ ]:
print(gender_bias)

[[0.7 0.3]
 [0.4 0.6]
 [0.5 0.5]]


## **Explanation of the Code:**
**Initialization:**

- The gender bias for each breeding pair is set initially ```(gender_bias)```.
- Progeny genders are randomly generated ```(progeny_gender)```, where 1 represents male and 0 represents female.
- The initial assignments ```(assignments)``` of progeny to each pair can be initialized one of two ways
  - randomly assigning each progeny to a breeding pair, using a Dirichlet distribution to ensure they sum to 1 for each progeny, or
  - assign equal probability for belonging to any breeding pair

**E-step:**

- The function e_step calculates the probability that each progeny belongs to each pair based on their gender and the current gender bias for each pair. These probabilities are normalized for each progeny to sum to 1 across all pairs.

**M-step:**

The function `m_step` updates the gender bias for each breeding pair based on the current assignments. It calculates the total expected number of male and female progeny assigned to each pair and uses that to update the gender bias estimates.

**EM Loop:**

The `em_algorithm` function iterates between the `E-step` and `M-step` until the change in gender bias estimates is **small enough (less than the tolerance `tol`)**, indicating convergence.

After convergence, it returns the final assignments and updated gender bias estimates.

Lets modify our scenario so that, we do not really know what the gender bias of each breeding pair is, but we know that there are 3 of them, and we have a collection of 60 progeny coming from these breeding pairs, organized in sets of 10.


The goal is to estimate both the gender biases and the progeny assignments using the Expectation-Maximization (EM) algorithm.

In [ ]:
import numpy as np

# Parameters
num_progeny = 60
num_pairs = 3
progeny_per_set = 10
num_sets = num_progeny // progeny_per_set

# Gender of progeny: 1 for male, 0 for female (simulated data: 60 progeny in sets of 10)
progeny_gender = np.random.choice([1, 0], size=num_progeny, p=[0.5, 0.5])

# Initial random assignment probabilities for each progeny to each pair
#assignments = np.random.dirichlet(np.ones(num_pairs), num_progeny)

# or initial assignment of equal chance to belong to any breeding pair
assignments = np.full((num_progeny, num_pairs), 1 / num_pairs)

# Initial random gender biases for each pair (P(male), P(female) for each pair)
gender_bias = np.random.rand(num_pairs, 2)
gender_bias[:, 1] = 1 - gender_bias[:, 0]  # Ensure P(male) + P(female) = 1

# E-step: Compute the probability that each progeny belongs to each pair
def e_step(progeny_gender, assignments, gender_bias):
    for i in range(num_progeny):
        for k in range(num_pairs):
            if progeny_gender[i] == 1:  # Male progeny
                assignments[i, k] = gender_bias[k, 0]  # P(male | pair k)
            else:  # Female progeny
                assignments[i, k] = gender_bias[k, 1]  # P(female | pair k)
        # Normalize to ensure the probabilities sum to 1
        assignments[i, :] /= np.sum(assignments[i, :])
    return assignments

# M-step: Update the gender bias and proportion estimates based on assignments
def m_step(assignments, progeny_gender):
    updated_gender_bias = np.zeros((num_pairs, 2))

    # Update gender bias for each pair
    for k in range(num_pairs):
        total_male = np.sum(assignments[:, k] * (progeny_gender == 1))
        total_female = np.sum(assignments[:, k] * (progeny_gender == 0))
        total_progeny = total_male + total_female

        updated_gender_bias[k, 0] = total_male / total_progeny  # P(male | pair k)
        updated_gender_bias[k, 1] = total_female / total_progeny  # P(female | pair k)

    return updated_gender_bias

# EM Algorithm Loop
def em_algorithm(progeny_gender, assignments, gender_bias, max_iter=100, tol=1e-6):
    for iteration in range(max_iter):
        # E-step: Calculate the expected assignment probabilities
        assignments = e_step(progeny_gender, assignments, gender_bias)

        # M-step: Update the gender bias estimates
        new_gender_bias = m_step(assignments, progeny_gender)

        # Check for convergence
        if np.max(np.abs(gender_bias - new_gender_bias)) < tol:
            print(f"Converged after {iteration} iterations")
            break

        gender_bias = new_gender_bias

    return assignments, gender_bias

# Run the EM algorithm
final_assignments, final_gender_bias = em_algorithm(progeny_gender, assignments, gender_bias)

# Results: Final assignment probabilities and gender biases
print("Final Gender Bias Estimates:")
print(final_gender_bias)

#print("Final Progeny Assignments (Probabilities):")
#print(final_assignments)


Converged after 16 iterations
Final Gender Bias Estimates:
[[0.23729905 0.76270095]
 [0.94062469 0.05937531]
 [0.22207215 0.77792785]]


To compare the real (true) probabilities against the estimated probabilities from the EM algorithm, you need to have two things:

**Real/True Gender Bias and Progeny Assignments:** This represents the actual gender bias for each pair and the actual pair that produced each progeny. This information can be generated in the case of a simulation or available in real-life data.

**Estimated Gender Bias and Progeny Assignments:** This is the output of the EM algorithm, which provides the estimated gender biases and the soft assignments of progeny to breeding pairs.

Lets revise the code again to enable this comparison

In [ ]:
import numpy as np

# Parameters
num_progeny = 60
num_pairs = 3
progeny_per_set = 10
num_sets = num_progeny // progeny_per_set

# Step 1: Generate real data for comparison
# True gender biases for each pair (P(male), P(female) for each pair)
true_gender_bias = np.array([[0.7, 0.3],  # Pair 1: 70% male
                             [0.4, 0.6],  # Pair 2: 40% male
                             [0.5, 0.5]]) # Pair 3: 50% male

# True progeny assignments: Assign each progeny to a specific pair
true_assignments = np.random.choice(num_pairs, size=num_progeny)

# Generate the gender of progeny based on true assignment and gender bias
progeny_gender = np.array([np.random.choice([1, 0], p=true_gender_bias[true_assignments[i]])
                           for i in range(num_progeny)])

# Step 2:
# Initial random assignment probabilities for each progeny to each pair
#assignments = np.random.dirichlet(np.ones(num_pairs), num_progeny)

# or initial assignment of equal chance to belong to any breeding pair
assignments = np.full((num_progeny, num_pairs), 1 / num_pairs)

# Random initial gender biases
gender_bias = np.random.rand(num_pairs, 2)
gender_bias[:, 1] = 1 - gender_bias[:, 0]  # Ensure P(male) + P(female) = 1

# E-step: Compute the probability that each progeny belongs to each pair
def e_step(progeny_gender, assignments, gender_bias):
    for i in range(num_progeny):
        for k in range(num_pairs):
            if progeny_gender[i] == 1:  # Male progeny
                assignments[i, k] = gender_bias[k, 0]  # P(male | pair k)
            else:  # Female progeny
                assignments[i, k] = gender_bias[k, 1]  # P(female | pair k)
        # Normalize to ensure the probabilities sum to 1
        assignments[i, :] /= np.sum(assignments[i, :])
    return assignments

# M-step: Update the gender bias and proportion estimates based on assignments
def m_step(assignments, progeny_gender):
    updated_gender_bias = np.zeros((num_pairs, 2))

    # Update gender bias for each pair
    for k in range(num_pairs):
        total_male = np.sum(assignments[:, k] * (progeny_gender == 1))
        total_female = np.sum(assignments[:, k] * (progeny_gender == 0))
        total_progeny = total_male + total_female

        updated_gender_bias[k, 0] = total_male / total_progeny  # P(male | pair k)
        updated_gender_bias[k, 1] = total_female / total_progeny  # P(female | pair k)

    return updated_gender_bias

# EM Algorithm Loop
def em_algorithm(progeny_gender, assignments, gender_bias, max_iter=100, tol=1e-6):
    for iteration in range(max_iter):
        # E-step: Calculate the expected assignment probabilities
        assignments = e_step(progeny_gender, assignments, gender_bias)

        # M-step: Update the gender bias estimates
        new_gender_bias = m_step(assignments, progeny_gender)

        # Check for convergence
        if np.max(np.abs(gender_bias - new_gender_bias)) < tol:
            print(f"Converged after {iteration} iterations")
            break

        gender_bias = new_gender_bias

    return assignments, gender_bias

# Run the EM algorithm
final_assignments, final_gender_bias = em_algorithm(progeny_gender, assignments, gender_bias)

# Step 3: Compare Real vs. Estimated Gender Bias
print("Real Gender Biases:")
print(true_gender_bias)

print("Estimated Gender Biases:")
print(final_gender_bias)

# Step 4: Compare Real vs. Estimated Progeny Assignments
# For each progeny, we will take the most likely pair from the final assignments
estimated_assignments = np.argmax(final_assignments, axis=1)

# Print comparison
# print("\nReal Progeny Assignments vs Estimated Assignments (most probable):")
# for i in range(num_progeny):
#     print(f"Progeny {i+1}: Real Pair {true_assignments[i]}, Estimated Pair {estimated_assignments[i]}")

# Calculate accuracy of the estimated assignments
accuracy = np.mean(true_assignments == estimated_assignments)
print(f"\nAssignment Accuracy: {accuracy * 100:.2f}%")


Converged after 9 iterations
Real Gender Biases:
[[0.7 0.3]
 [0.4 0.6]
 [0.5 0.5]]
Estimated Gender Biases:
[[0.68268766 0.31731234]
 [0.57401403 0.42598597]
 [0.09329691 0.90670309]]

Assignment Accuracy: 33.33%


That is not very accurate, is it?

**How can we improve it? **